## Summary

This tutorial uses napari 0.4.11. 

Dataset (flyLight):<br>
[Image file download link](https://drive.google.com/file/d/1CijVxebE4a-mMMvkB7HxuVdFHvO378Eg/view?usp=sharing).<br> 
Original file downloaded from https://www.janelia.org/project-team/flylight<br>
3D, 4 channel

Analysis goal:<br> 
Measure the length of main neuron from ch2

Tutorial goals:<br>
(1) Use napari + Python libraries to perform common image analysis from end to end.<br>
(2) Note napari viewer GUI function availability at varioius steps.  

Steps:
- Load 3D multi-channel image (flyLight)
- Get image dimension and display with correct axes
- Adjust image display
- Crop and downsample image
- Denoising by median filter
- Segmentation by simple threshold
- Clean up segmentation result
- Set pixel size
- Save segmented result: labels and properties

<font color=red> RED: links to add<br> </font>
<font color=blue> BLUE: GUI availability in viewer </font>

In [ ]:
import napari
from napari.utils import nbscreenshot
viewer = napari.Viewer()

## Load image
napari natively supports tiff (and ???) as input image file format.<br>
Additional input file formats may be supported by plugins.<br>

<font color=blue> GUI has several file open options (drag and drop, File > Open File) <br></font>
<font color=red>link: supported file format and plugins</font>

In [ ]:
from tifffile import imread
#modify the file path to match file location
flyLight = imread('/Users/cchiu/Desktop/images/flyLight_aligned_stack.tif')
print("original file dimension: ", flyLight.shape)

## Get image dimension and display with correct axes
The original shape is (219, 4, 1119, 573), corresponding to axes order ZCYX.<br>
napari takes the axes order TZYX with C represented as separate image layers.<br> 
To display the image correctly, use np.transpose to get the dimensions in the right order, then split each channel as one image layer for easier display/analysis.<br>
<font color=blue>
    Changing axes is challenging to do in GUI<br>
    Channel name and ndisplay can be changed in GUI
</font>


In [ ]:
import numpy as np
#chanage the axes to match CZYX
flyLight_dim_correct = np.transpose(flyLight, (1,0,2,3))

#load multi-channel image in one line, with channel name, and display in 3D
viewer = napari.view_image(flyLight_dim_correct, channel_axis=0, name=["ch1","ch2","ch3","ch4"], ndisplay=3)

#equivalent to
#for i in range(flyLight_dim_correct.shape[0]):
#    viewer.add_image(np.squeeze(flyLight_dim_correct[i,:,:,:]))    

## Adjust image display

<font color=blue> Both API and GUI have flexible image display control. <blue>    
<font color=red>Link: API and GUI display reference</font>


In [ ]:
#change ch1 color from cyan to gray 
viewer.layers['ch1'].colormap = 'gray'
#change ch4 opacity to 0.6
viewer.layers['ch4'].opacity = 0.6

In [ ]:
nbscreenshot(viewer)

## Crop and downsample image
<br>
<font color=blue>
    No crop and downsample funtions in GUI.<br>
    Cursor coordinate (in pixel) is displayed in GUI, which helps with determining ROI.   
</font>


In [ ]:
#original image flyLight_dim_correct has CZYX size (4,219,1119,573)
print("original image size: ", flyLight_dim_correct.shape)

#crop x and y 20 pixels from each side, and downsample 2x along z
flyLight_crop = flyLight_dim_correct[:,::2,20:1098,20:552]

#remove the original image layers
viewer.layers.select_all()
viewer.layers.remove_selected()

#load cropped image
viewer.add_image(flyLight_crop, channel_axis=0, name=["ch1","ch2","ch3","ch4"])

#new image ZYX size
print("new image size: ", viewer.layers[0].data.shape)

In [ ]:
nbscreenshot(viewer)

## Denoising by median filter

To reduce noise in channel 2:<br> 
(1) Apply median filter to remove salt-and-pepper noise<br>
(2) Create a new image layer for the filtered image

<font color=blue>Available plugin: pyclesperanto<br></font>
<font color=red>link: magicgui to make interactive plot<br>
</font>

In [ ]:
import skimage
#median filter
filtered_ch2 = skimage.filters.median(viewer.layers["ch2"].data)
viewer.add_image(filtered_ch2, name='filtered ch2')

In [ ]:
nbscreenshot(viewer)

## Segmentation by simple threshold

(1) Use intensity histogram to determine the appropriate intensity threshold<br>
(2) Use the intensity threshold to create a binarized image<br>
(3) Convert the binarized image to label (i.e. segmented and represented as unique integer per object)<br>

<br>
<font color=blue>
    No intensity histogram in GUI.<br>
    Label supports manual annotation in GUI.<br>
</font>
<font color=red>link: segmentation plugins and labels layer</font>

In [ ]:
#plot intensity distribution in ch2
from matplotlib.pyplot import hist
hist(filtered_ch2.flatten(),bins=50,log=True)

In [ ]:
#binarize the image using intensity = 150
threshold = 150
cutoff_ch2 = filtered_ch2 > threshold
viewer.add_image(cutoff_ch2, name='cutoff ch2')

#turn binary image into label layer
seg_ch2 = skimage.measure.label(cutoff_ch2)
viewer.add_labels(seg_ch2, name='ch2 seg')

In [ ]:
nbscreenshot(viewer)

## Clean up segmentation result
(1) [not shown here] Use plugin segmentation (split/merge) to manually merge the neuron segments<br>
(2) Remove small objects

<font color=blue>
    Label can be interactively modified in GUI.<br> 
    No object property GUI.<br></font>
<font color=red>link: feature extraction plugins - as of now show the property table but no selection tool, and may have less option for 3D.</font>

In [ ]:
props = skimage.measure.regionprops_table(seg_ch2, properties=['label','area'])

import pandas as pd
#object property (area) distribution
data = pd.DataFrame(props)
hist(data['area'],bins=10,log=True)

In [ ]:
#create new label layer with only larger objects
bigger_obj_label = skimage.morphology.remove_small_objects(seg_ch2, min_size=1000)
viewer.add_labels(bigger_obj_label,name='bigger objects')

In [ ]:
# only show the bigger object layer
for layer in viewer.layers:
    layer.visible = False
viewer.layers['bigger objects'].visible = True

nbscreenshot(viewer)

## Set pixel size
Default pixel size is set to (1,1,1) unless directly read from metadata.<br> 
Pixel size affects visualization scaling, remember to adjust the value if downsampled.<br>

<font color=blue> 
    No GUI to specify pixel size and time interval.
</font>

In [ ]:
#set pixel size in um scale (z,y,x)
pixel_size = np.array([2,1,1])

for img_layer in viewer.layers:
    img_layer.scale = pixel_size

## Save segmented result
Label layer export<br>
Option 1: save as numpy .npy format - can be opened directly by viewer<br>
Option 2: save as tiff - the export format when saved from GUI<br>
*Note that properties (pixel size etc.) are not saved and need to be specified again when opening the saved file.

Object property export<br>
Option 1: save object properties as pickle<br>
Option 2: save object properties as csv<br> 

<font color=blue>
    GUI supports File > Save Layer.<br>
    Feature extraction plugin provides csv export.<br>
</font>
<font color=red>link: label IO plugins<br>

In [ ]:
#save label layer in numpy .npy format 
import numpy
numpy.save('FlyLight_label', bigger_obj_label)

#save label layer in tiff format
viewer.layers['bigger objects'].save('FlyLight_label.tiff')

In [ ]:
#save object properties (pnadas dataframe) as pickle
data.to_pickle('flyLight_property.pkl')

#save object properties as csv
data.to_csv('flyLight_property.csv')